In [1]:
#------------------------------importing library---------------------------------------
from dateutil.parser import parse
import numpy as np
import pandas as pd
import pickle
# from bpemb import BPEmb
import pytesseract
import cv2
import glob
%matplotlib notebook
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
import re
from timeit import default_timer as timer
import shutil

from xml.etree import ElementTree
from xml.dom import minidom
import xml.etree.cElementTree as ET 

In [2]:
text_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/*.txt"
img_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/*.jpg"
tesseract_path = 'C:/python_lib/ocr/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = 'C:/python_lib/ocr/tesseract.exe'
file_save_path='C:/Users/Sanjeev/Documents/Python Scripts/Dataset/label_data/'
xml_file="C:/Users/Sanjeev/Documents/Python Scripts/NLP/TensorFlow/addons/data/X00016469612.xml"
img_save_path='C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/generated_annotation/'
text_save_path='C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/data/generated_annotation/text_dict/'

In [3]:
text_path_list = glob.glob(text_path)
img_path_list = glob.glob(img_path)

In [4]:
def SaveData(data,path):
    with open(path,'wb') as file:
        pickle.dump(data,file)

def LoadData(path):
    with open(path, 'rb') as file:
        return pickle.load(file)

def display_image(img):
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    if len(img.shape) > 2:
        plt.imshow(img, cmap='gray')
        plt.title("Image")
        plt.axis('off')
    else:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title("RGB Image")
        plt.axis('off')

        
def extract_bbox_info(img_path):
    hocr_data = pytesseract.image_to_pdf_or_hocr(img_path, extension='hocr')
    soup = BeautifulSoup(hocr_data)
    spans = soup.find_all('span', attrs={'class':'ocrx_word'})  # getting the attributes 
    word_bbox_list =[]  
#     bbox_width=[]
    for span in spans:
        bbox_str=span['title']  # geting bounding box inforamtion 
        bbox_str =bbox_str.split(';')
        bbox =bbox_str[0].split()[1:]
    #     x,y,w,h= *bbox
    #     word_info_list.append([span['id'],span.string,[bbox[1].split()[-1],*bbox]])
        word_bbox_list.append([span['id'],span.string,
                               [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                              ])
#         bbox_width.append((int(bbox[2])-int(bbox[0])))
#         word_info_list.sort(key = lambda sort_value: (sort_value[2][1],sort_value[2][0]))
#     word_info_list.sort(key = lambda sort_value: sort_value[2][1])
    
    return word_bbox_list

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ") 
# from ElementTree_pretty import prettify
def create_xml(bbox_list=None,path=None,img_shape=None,folder='Data'):
    w,h,d=img_shape
    
    root = ET.Element("annotation")
    root_c_folder = ET.SubElement(root, "folder").text =folder
    root_c_filename=ET.SubElement(root, "filename").text =path.split('\\')[-1]
    
    root_c_path=ET.SubElement(root, "path").text =path
    root_c_source=ET.SubElement(root, "source")
    ET.SubElement(root_c_source, "database").text = "Unknown"
    
    root_c_size=ET.SubElement(root, "size")
    ET.SubElement(root_c_size, "width").text = str(int(w))
    ET.SubElement(root_c_size, "height").text = str(int(h))
    ET.SubElement(root_c_size, "depth").text = str(int(d))
    
    root_c_segmented=ET.SubElement(root, "segmented").text=str(int(0))
    
    for bbox_ in bbox_list:
        bbox =bbox_[-1]
        root_c_object=ET.SubElement(root, "object")

        ET.SubElement(root_c_object, "name").text = 'text'
        ET.SubElement(root_c_object, "pose").text = 'Unspecified'
        ET.SubElement(root_c_object, "truncated").text = str(int(0))
        ET.SubElement(root_c_object, "difficult").text = str(int(0))
        object_c_bndbox=ET.SubElement(root_c_object, "bndbox")
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[0]))
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[1]))
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[2]))
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[3]))

    return prettify(root)
#     tree.write(file_save_path+"filename.xml")

In [5]:
def get_ocr_text(img):  
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    text = pytesseract.image_to_string(img)
#     print(text)
    return text

# read_data ()-> out data [ocr_text_list,file_name_list,word_list,dict_list,img_list,path_list]
def read_data_xml(img_path_list, text_path_list, data_no=150):
    word_list,dict_list,img_list,path_list,file_name_list,ocr_text_list=[],[],[],[],[],[]
    
    for i in range(data_no):
        print('file : ',i, '-> remaining : ',data_no-i)
        img_path= img_path_list[i]
        path_list.append(img_path_list[i])
        img_name = img_path.split("\\")[-1]
        img_name = img_name.split(".jpg")[0]
        img_data = cv2.imread(img_path_list[i])
        img_list.append(img_data)
        ocr_text = get_ocr_text(img_path)
        ocr_text_list.append(ocr_text)
        word_list.append(extract_bbox_info(img_path_list[i]))
        text_path = text_path_list[i]
        text_name=text_path.split("\\")[-1]
        text_name = text_name.split('.txt')[0]
        while img_name != text_name:
            j=i 
            if img_name.split !=text_name.split(" ")[0]:
                break
            else:

                j +=1
                print('fileName is not same: ',img_name, text_name)
                text_path = text_path_list[j]
                text_name=text_path.split("\\")[-1]
                text_name = text_name.split('.txt')[0]
        with open(text_path, 'r') as file:
            text_data =file.read()
        dict_list.append(eval(text_data))
        file_name_list.append(img_name)
#         data_list.append([img_data,ocr_text,eval(text_data),img_name])    
    return ocr_text_list,file_name_list,word_list,dict_list,img_list,path_list

def read_data_yolo(img_path_list, text_path_list, data_no=150):
    word_list,dict_list,img_list,path_list,file_name_list,ocr_text_list=[],[],[],[],[],[]
    for i in range(data_no):
        print('file : ',i, '-> remaining : ',data_no-i)
        img_path= img_path_list[i]
        path_list.append(img_path_list[i])
        img_name = img_path.split("\\")[-1]
        img_name = img_name.split(".jpg")[0]
        img_data = cv2.imread(img_path_list[i])
        img_list.append(img_data)
        ocr_text = get_ocr_text(img_path)
        ocr_text_list.append(ocr_text)
        word_list.append(extract_bbox_info(img_path_list[i]))
        text_path = text_path_list[i]
        text_name=text_path.split("\\")[-1]
        text_name = text_name.split('.txt')[0]
        with open(text_path, 'r') as file:
            text_data =file.read()
        dict_list.append(eval(text_data))
        file_name_list.append(img_name)
#         data_list.append([img_data,ocr_text,eval(text_data),img_name])    
    return ocr_text_list,file_name_list,word_list,dict_list,img_list,path_list

In [6]:
def Line_Formation(word_list):
    word_list.sort(key =lambda sort_on_top : sort_on_top[2][1]) # sorting on top value 
    line_list =[]  
    word_list_len = len(word_list)
    temp=[]
    temp.append(word_list[0])   # intializing the value with pre-assumption 
    for i in range(1,word_list_len, 1):
        wa_top =word_list[i-1][2][1]
        wa_bottom = word_list[i-1][2][3]

        wb_top =word_list[i][2][1]
        wb_bottom = word_list[i][2][3]
        if wa_top <= wb_bottom and wa_bottom >= wb_top:
            temp.append(word_list[i])
        else:
            temp.sort(key=lambda sort_on_left_ : sort_on_left_[2][0]) # sorting on the left value
            line_list.append(temp)
            temp=[]
            temp.append(word_list[i])
    line_list.append(temp)
    return line_list

# output-> arr_word_list, dict_list
def create_output(word_list,dict_word,skip=None):
    dict_list = []
    for key,value in dict_word.items():
        if skip==key:
            continue
        dict_list.append(value.split())

        arr_word_list=[]
    lines=Line_Formation(word_list)
    for line in lines:
        for word in line:
            arr_word_list.append(word)
        
    return arr_word_list, dict_list

In [7]:
def get_index(text,dict_list):
    for idx,word_list in enumerate(dict_list):
        if text in word_list:
            return idx
    return -1

def get_bbox_list(arr_word_list,dict_list):
    bbox_list,idx =[],0
    while idx < len(arr_word_list):
        text= arr_word_list[idx][1]
        value_idx = get_index(text,dict_list)
        if value_idx ==-1:
            idx +=1
            continue
#         print('value_id : ',value_idx,'idx ', idx, 'text = ', text)
#         print(text,"--->", dict_list[value_idx])
        if text in dict_list[value_idx]:     #case when 1st word matches
            l=dict_list[value_idx].index(text) # if index is not zero
#             print('-------> index check (',text,'): ', l )
            j=0
            if l != 0:
#                 print('case when l not eaual to zero vlaue l ', l)
                j=l
                while l > 0:
#                     print('appending word: ',arr_word_list[idx-l])
                    bbox_list.append(arr_word_list[idx-l])
                    l=l-1
            while j < len(dict_list[value_idx]) and arr_word_list[idx][1] != dict_list[value_idx][-1]:  
                bbox_list.append(arr_word_list[idx])  # todo what should be the output data actural data or extracted one (now). 
                idx +=1
                j +=1
            bbox_list.append(arr_word_list[idx])
            del dict_list[value_idx]
            value_idx +=1
        idx +=1
    return bbox_list
    

In [8]:
_,_,word_list,dict_list,img_list,path_list = read_data_xml(img_path_list, text_path_list, data_no=2)

reading image ../n
reading image ../n
fileName is not same:  X00016469619 X00016469612


In [8]:
def create_xml_annotation(word_list, dict_list, img_list, path_list, file_save_path=''):
    
    file_len =len(word_list)
    
    for idx in range(file_len):
        
        arr_word_list, dict_text_list = create_output(word_list[idx],dict_list[idx])
        bbox_list = get_bbox_list(arr_word_list,dict_text_list)

        xml_text=create_xml(bbox_list,path_list[idx],img_list[idx].shape,folder='Data')
        file_name = path_list[idx].split('\\')[-1]
        file_name=file_name.split('.')[0]+'.xml'
        with open(file_save_path+file_name, "w") as file:
            file.write(xml_text)
        print(xml_text,'\n\n---------------------------------\n\n')

In [9]:
create_xml_annotation(word_list, dict_list, img_list, path_list,file_save_path)

NameError: name 'word_list' is not defined

In [11]:
img_save_path =img_save_path+'temp/'


In [12]:
# copy_data ()-> out data [image_list,ocr_text,out_text,file_name]
def copy_data(img_path_list, text_path_list,img_save_path, data_no=150):
    img_list,ocr_text_list,text_list,file_name_list=[],[],[],[]
    
    for i in range(data_no):
        img_path= img_path_list[i]
        img_name = img_path.split("\\")[-1]
        shutil.copyfile(img_path, img_save_path+img_path.split("\\")[-1])
#         img_name=img_name.split('.')[0]
#         text_path = text_path_list[i]
#         text_name=text_path.split("\\")[-1]
#         text_name = text_name.split('.txt')[0]
#         while img_name != text_name:
#             j=i
#             if img_name.split !=text_name.split(" ")[0]:
#                 break
#             else:
#                 j +=1
#                 print('fileName is not same: ',img_name, text_name)
#                 text_path = text_path_list[j]
#                 text_name=text_path.split("\\")[-1]
#                 text_name = text_name.split('.txt')[0]
#         print(i,img_name,text_name) 
#         shutil.copyfile(text_path, text_save_path+text_path.split("\\")[-1])

In [13]:
copy_data(img_path_list,text_path_list,img_save_path,200)

In [35]:
text_path_list = glob.glob(text_save_path + '*.txt')
img_path_list = glob.glob(img_save_path+'*.jpg')
print(len(text_path_list),len(text_path_list))

200 200


In [64]:
_,_,word_list,dict_list,img_list,path_list = read_data_yolo(img_path_list, text_path_list, data_no=200)

reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading image ../n
reading imag

In [66]:
def create_yolo_annotation(word_list, dict_list, img_list, path_list, file_save_path=''):
    
    file_len =len(word_list)
    class_id =0
    
    for idx in range(file_len):
        print('file : ',idx, '-> remaining : ',200-idx)
        arr_word_list, dict_text_list = create_output(word_list[idx],dict_list[idx],skip='total')
        bbox_list = get_bbox_list(arr_word_list,dict_text_list)
        X,Y,_ =img_list[idx].shape
        file_name = path_list[idx].split("\\")[-1]
        file_name=file_name.split('.')[0]
#         print(file_save_path +file_name+'.txt')
        with open(file_save_path +file_name+'.txt', 'w') as file:
            
            for bbox in bbox_list:
                x,y,w,h=bbox[-1]
                x0=round((x+(w-x)/2)/Y, 6)
                y0=round((y+(h-y)/2)/X, 6)
                W=round((w-x)/Y, 6)
                H=round((h-y)/X, 6)
                file.writelines(str(class_id)+' '+str(x0)+' '+str(y0)+' '+str(W)+' '+str(H)+'\n')
            #             print('x={},y={},W={},h={}'.format(bbox[-1][0],bbox[-1][1],bbox[-1][2],bbox[-1][3]))
#             print('w-x(x2)={},h-y(y2)={}'.format(bbox[-1][2] - bbox[-1][0], bbox[-1][3]- bbox[-1][1]))
#             print('u-> w-x(x2)={},h-y(y2)={}'.format(bbox[-1][0] - bbox[-1][2], bbox[-1][1]- bbox[-1][3]))
# #             print('w-x(W)={},h-y(H)={}'.format((bbox[-1][2] - bbox[-1][0])/X, (bbox[-1][3]- bbox[-1][1]))/Y)
#             print('{} {} {} {} {}'.format(class_id,x0,y0,W,H)) 
#             break
#         print('\n-------------next\n')

In [67]:
create_yolo_annotation(word_list, dict_list, img_list, path_list,img_save_path)

file :  0 -> remaining :  200
file :  1 -> remaining :  199
file :  2 -> remaining :  198
file :  3 -> remaining :  197
file :  4 -> remaining :  196
file :  5 -> remaining :  195
file :  6 -> remaining :  194
file :  7 -> remaining :  193
file :  8 -> remaining :  192
file :  9 -> remaining :  191
file :  10 -> remaining :  190
file :  11 -> remaining :  189
file :  12 -> remaining :  188
file :  13 -> remaining :  187
file :  14 -> remaining :  186
file :  15 -> remaining :  185
file :  16 -> remaining :  184
file :  17 -> remaining :  183
file :  18 -> remaining :  182
file :  19 -> remaining :  181
file :  20 -> remaining :  180
file :  21 -> remaining :  179
file :  22 -> remaining :  178
file :  23 -> remaining :  177
file :  24 -> remaining :  176
file :  25 -> remaining :  175
file :  26 -> remaining :  174
file :  27 -> remaining :  173
file :  28 -> remaining :  172
file :  29 -> remaining :  171
file :  30 -> remaining :  170
file :  31 -> remaining :  169
file :  32 -> rema

In [ ]:
%debug

In [ ]:
print(52/463)
print((109-97)/1013)


In [ ]:
xml_f="C:/Users/Sanjeev/Documents/Python Scripts/Dataset/label_data/X00016469612.xml"

In [ ]:
from xml.dom import minidom

# parse an xml file by name
mydoc = minidom.parse(xml_f)

items = mydoc.getElementsByTagName('object')

In [ ]:
items

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse(xml_f)
root = tree.getroot()
print('\nAll item data:')
for elem in root:
    for subelem in elem:
        print(subelem.text)
        for bbox in subelem:
            print('bbox :-> ',bbox.text)


In [ ]:
text_path_list = glob.glob(text_path)
img_path_list = glob.glob(img_path)
print(len(text_path_list) , len(img_path_list))

In [ ]:
for i in range(len(img_path_list)):
    if img_path_list[i].split("\\")[0] == text_path_list[i].split("\\")[0]:
        continue
    else:
        print('data is not same for : ',i ," \n",img_path_list[i].split("\\")[0]," \n",text_path_list[i].split("\\")[0])